In [1]:
import cv2

In [2]:
def cv2_imshow(image):
    cv2.imshow("",image)
    k = cv2.waitKey(0) & 0xFF
    if(k==27):
        cv2.destroyAllWindows()

In [22]:
import base64
# import HTML
import html
from IPython.display import HTML
from base64 import b64encode
# from weasyprint import HTML
def show_video(video_path):
    video_file = open(video_path,"r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video width=600 controls><source src="{video_url}"></video>""")

In [4]:
video_path = "Road - 1101.mp4"

In [5]:
capture = cv2.VideoCapture(video_path)

In [6]:
capture

<VideoCapture 0x7f8b39a6f0>

In [7]:
import numpy
rectangle = numpy.zeros((300,300), dtype="uint8")
cv2.rectangle (rectangle,(10,10),(200,200),255,-1)
# cv2_imshow(rectangle)



array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [8]:
import numpy
circle = numpy.zeros((300,300), dtype ="uint8" )
cv2.circle(circle, (150,150),100,255,-1)
# cv2_imshow(circle)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [9]:
result = cv2.bitwise_and(rectangle,circle)
# cv2_imshow(result)

In [10]:
import numpy
def find_region_of_interest(edges, vertices):
    mask = numpy.zeros_like(edges)
    mask_color = 255
    new_image = cv2.fillPoly(mask, vertices, mask_color)
    bitwise_mask = cv2.bitwise_and(edges, mask)
#     cv2_imshow(bitwise_mask)
    return bitwise_mask

In [11]:
import numpy


def  draw_lane_lines(frame, lines):
    blank_image = numpy.zeros((frame.shape[0],
                              frame.shape[1],
                              3),
                             dtype=numpy.uint8)
    color = (255,255,0)
    thickness = 2 
    frame_weight = 0.8
    blank_image_weight = 1
    gamma = 0.0
    for line in lines:
        for start_x, start_y, end_x, end_y in line:
            cv2.line(blank_image, (start_x, start_y),
                    (end_x, end_y), color, thickness)
            
    lane_image = cv2.addWeighted(frame,frame_weight, blank_image, blank_image_weight, gamma) 
    
    frame_list.append(lane_image)
            
#     cv2_imshow(lane_image)        

In [12]:
import numpy
def process_frame(frame):
    gray_image = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
   
    threshold1 = 100
    threshold2 = 120
    edges = cv2.Canny(gray_image, threshold1, threshold2)
    
    height, width, color = frame.shape
    custom_size = 100
    custom_x = 1.9
    vertices = [(0, height - custom_size),
                (width/custom_x, height/custom_x),
                (width, height - custom_size)]
    
    cropped_image = find_region_of_interest(edges, 
                                            numpy.array([vertices], 
                                                        numpy.int32))
    lines = cv2.HoughLinesP(cropped_image,
                  rho = 1,
                  theta = numpy.pi/180,
                  threshold = 90,
                  lines = numpy.array([]),
                  minLineLength = 0.5,
                  
                  maxLineGap = 500)
    
    draw_lane_lines(frame, lines)
    
#     cv2_imshow(gray_image)
#     cv2_imshow(edges)

In [13]:
number_of_frames = 30
frame_list = []
for i in range(number_of_frames):
    ret, frame = capture.read()
    process_frame(frame)
    
    height, width, color = frame.shape
    size = (width,height)
#     cv2_imshow(frame)
capture.release()    

In [18]:
output_path = "LanesDetected.mp4"
frames_per_second = 30
output = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"DIVX"),
               frames_per_second,
               size)

for frame in range(len(frame_list)):
    output.write(frame_list[frame])
    
output.release()    
    

In [21]:
show_video(output_path)